# SP702 Final Project (Task 5)

**By**: Francis Mark M. Cayco

**Github profile**: https://github.com/PeteCastle

**Instructions:**

*Given the location you considered in task 4, extract location data. o Location Name (Note: Barangay, City/Municipality, Province)*
- Population (Note: Population must be in thousands)
- Available transportation
- Reviews/description of location
- Primary language
- Secondary language
- Other data (extract more data regarding tourist sites that can be used by tourists) 

Store the collected data in a csv file. File name format: **location_source.csv** -> **I have corrected the error in the file name. Please use this format instead.**

Please upload your Jupyter notebook file. 
Filename format:
Extract_Location_Source.ipynb. Note that you might need to create different python code per source of data. Submit all code you used to extract data.

## Data Sources
1. Philippine Statistics Authority OpenStat (https://openstat.psa.gov.ph/)
2. Humanitarian Data Exchange by United Nations Population Fund (https://data.humdata.org/)

## What are the attributes of the data that I have extracted?
1. Population  (in thousands)
2. Languages (primary to tertiary)
3. Transportation
4. Gross Regional Domestic Product (GRDP)
5. Consumer Price Index (CPI)

All barangay, city/municipality, and province in the Philippines are included in the data.

## List of columns for final table
File is saved in **location_source.csv**.

| **Basic Information** |           **Description**                |
|-----------------------|---------------------------|
| Barangay              | Barangay                  |
| City/Municipality     | City or Municipality     |
| Province              | Province of the Philippines |
| Region                | Region of the Philippines |
| 2020 Population       | Population in thousands  |
| Description           | Reviews/description of location |
|---------------|------------------------------------|
| **Languages** |                              |
| Primary_Language           | Most spoken language in the associated province |
| Primary_Language_Share     | Percentage of primary language contribution to total population |
| Secondary_Language         | Second most spoken language in the associated province |
| Secondary_Language_Share   | Percentage of secondary language contribution to total population |
| Tertiary_Language          | Third most spoken language in the associated province |
| Tertiary_Language_Share    | Percentage of tertiary language contribution to total population |
|---------------|------------------------------------|
| **Literacy Rates** |                              |
| Literacy_All    | Literacy rate of the province |
| Literacy_Male   | Male literacy rate of the province |
| Literacy_Female | Female literacy rate of the province |
|---------------|------------------------------------|
| **Economics** |                                    |
| GRDP_2022     | Gross Regional Domestic Product (GRDP) of the region |
| CPI_Alcoholic_Beverages_Tobacco | Consumer Price Index (CPI) of alcoholic beverages and tobacco |
| CPI_Clothing_Footwear | Consumer Price Index (CPI) of clothing and footwear |
| CPI_Education_Services | Consumer Price Index (CPI) of education services |
| CPI_Financial_Services | Consumer Price Index (CPI) of financial services |
| CPI_Food_Non-Alcoholic_Beverages | Consumer Price Index (CPI) of food and non-alcoholic beverages |
| CPI_Furnishings_Household_Equipment_Routine_Household_Maintenance | Consumer Price Index (CPI) of furnishings, household equipment, and routine household maintenance |
| CPI_Health | Consumer Price Index (CPI) of health |
| CPI_Housing_Water_Electricity_Gas_Other_Fuels | Consumer Price Index (CPI) of housing, water, electricity, gas, and other fuels |
| CPI_Information_Communication | Consumer Price Index (CPI) of information and communication |
| CPI_Personal_Care_Miscellaneous_Goods_Services | Consumer Price Index (CPI) of personal care, miscellaneous goods, and services |
| CPI_Recreation_Sport_Culture | Consumer Price Index (CPI) of recreation, sport, and culture |
| CPI_Restaurants_Accommodation_Services | Consumer Price Index (CPI) of restaurants and accommodation services |
| CPI_Transport | Consumer Price Index (CPI) of transport |
|---------------|------------------------------------|
| **Transportation** |                                |
| Regional_Inland_Water_Transport_Count | Number of inland water transport in the region |
| Regional_Air_Transport_Count | Number of air transport in the region |
| Regional_Coastal_Water_Transport_Count | Number of coastal water transport in the region |
| Regional_Bus_Transport_Count | Number of bus transport in the region |
| Regional_Railway_Transport_Count | Number of railway transport in the region |





<!-- 
# Income
https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__1B__OWS/0051B3E1050.px/table/tableViewLayout1/?rxid=89ed1f4a-0aef-4756-ae4d-b2b602466cbb -->

In [1]:
import os
import wget
import urllib
import pandas as pd
from collections import OrderedDict
from fuzzywuzzy import process
import numpy as np

### Downloading Population Data

**2020 Census of Population and Housing (2020 CPH) Population Counts Declared Official by the President**

Link: https://psa.gov.ph/content/2020-census-population-and-housing-2020-cph-population-counts-declared-official-president

In [2]:
raw_dataset_urls = [
    "https://psa.gov.ph/system/files/phcd/2022-12/NCR.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/CAR.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%25201.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%25202.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%25203.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%25205.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/%25281%2529%2520Region%25206_final.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%25207.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%25208.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%25209.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%252010.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%252011.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%252012.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Caraga.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/BARMM.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/Region%25204A.xlsx",
    "https://psa.gov.ph/system/files/phcd/2022-12/MIMAROPA.xlsx"
]

raw_data_directory = "raw_data/"
raw_files = []
for url in raw_dataset_urls:
    try:
        decodedURL = urllib.parse.unquote(url)
        fileName = decodedURL.split("/")[-1]
        raw_files.append(fileName)
        if not os.path.exists(f"{raw_data_directory}{fileName}"):
            print(f"Downloading url {url}")
            wget.download(decodedURL, out=f"{os.getcwd()}/raw_data")
        else:
            print(f"File {fileName} already exists")
    except urllib.error.HTTPError as e:
        print(f"An error has occurred while downloading url {url}: {e}") 

File NCR.xlsx already exists
File CAR.xlsx already exists
File Region%201.xlsx already exists
File Region%202.xlsx already exists
File Region%203.xlsx already exists
File Region%205.xlsx already exists
File %281%29%20Region%206_final.xlsx already exists
File Region%207.xlsx already exists
File Region%208.xlsx already exists
File Region%209.xlsx already exists
File Region%2010.xlsx already exists
File Region%2011.xlsx already exists
File Region%2012.xlsx already exists
File Caraga.xlsx already exists
File BARMM.xlsx already exists
File Region%204A.xlsx already exists
File MIMAROPA.xlsx already exists


In [3]:
population_dict = []
def appendRow(province : str, municipality : str, brgy_name : str, brgy_pop : str):
    global population_dict
    population_dict.append({'Province': province, 'Municipality' : municipality, 'Barangay' : brgy_name , '2020 Population' : brgy_pop})
  
def sanitizeString(s: str):
    s = s.replace("*",'')
    s = s.replace("(Pob.)",'')
    s = s.replace("(Capital)",'')
    s = s.strip()
    return s

In [4]:
# Independent cities
for file in raw_files:
    MANILA_DISTRICTS = {
        "TONDO I/II",
        "BINONDO",
        "QUIAPO",
        "SAN NICOLAS",
        "SANTA CRUZ",
        "SAMPALOC",
        "SAN MIGUEL",
        "ERMITA",
        "INTRAMUROS",
        "MALATE",
        "PACO",
        "PANDACAN",
        "PORT AREA"
        "SANTA ANA"
    }
    df_dictionary = OrderedDict(pd.read_excel(f"raw_data/{file}", sheet_name = None))
    df_dictionary.popitem(last=False) # Removes the first sheet

    print(f"Reading file {file}")
    column_idenifier = 'Total Population by Province, City, Municipality, and Barangay:'

    #Special Cases:
    if file == "Region%2012.xlsx": # for region 12
        df_dictionary.popitem(last=False)
    if file == "BARMM.xlsx" or file == "NCR.xlsx": # for BARMM and NCR
        column_idenifier = 'Total Population by Province, City, and Municipality:'

    for df_key in df_dictionary:
        df = df_dictionary[df_key]

        province_name : str =''
        municipality_name : str = ''
        barangay_name: str = ''
        barangay_population : int = 0

        name_col_index = df_dictionary[df_key].columns.get_loc(column_idenifier)
        pop_col_index = name_col_index+1

        for index, row in df.iterrows():
            name = sanitizeString(str(row.iloc[name_col_index]))
            if index < 5: 
                continue
            if str(name) == 'nan' :
                continue
            if str(name) in ["Note:","Source:","Notes:"]:
                break

            if index == 5:
                province_name = name.title()
            elif str(df.iloc[index-1][name_col_index]) == 'nan':
                #SPECIAL CASES FOR Manila
                if name in MANILA_DISTRICTS and province_name == "National Capital Region":
                    # print(f"Found a district in Manila: {name}")
                    municipality_name = name + ", Manila"
                else:
                    municipality_name = name.title()
            else:
                try:
                    barangay_name = name.title()
                    barangay_population = int(row[name_col_index+1])
                    # print(province_name, municipality_name, barangay_name, barangay_population)
                    appendRow(province_name, municipality_name, barangay_name, barangay_population)
                except ValueError:
                    print(f"{barangay_name} doesn't have a population count.  Removing from the database.")

population_data = pd.DataFrame(population_dict, columns = ['Province','Municipality','Barangay','2020 Population'])
population_data.to_csv('partial_data/population_data.csv', index=False)

Reading file NCR.xlsx
Reading file CAR.xlsx
(Quirino-Magsaysay, Lower) doesn't have a population count.  Removing from the database.
Reading file Region%201.xlsx
Reading file Region%202.xlsx
Reading file Region%203.xlsx
Reading file Region%205.xlsx
Reading file %281%29%20Region%206_final.xlsx
Reading file Region%207.xlsx
Reading file Region%208.xlsx
Reading file Region%209.xlsx
Reading file Region%2010.xlsx
(Don Mariano Marcos) doesn't have a population count.  Removing from the database.
Reading file Region%2011.xlsx
Reading file Region%2012.xlsx
Reading file Caraga.xlsx
Reading file BARMM.xlsx
Reading file Region%204A.xlsx
Reading file MIMAROPA.xlsx


### Obtaining Location Data
**Philippines - Subnational Administrative Boundaries**

https://data.humdata.org/dataset/cod-ab-phl?



<!-- OLD CODE

# def getCoordinatesWithPostal(municipality, province, country = "Philippines"): # and postal code
#     params = {
#         "q":f"{municipality}, {province}, {country}",
#         "polygon":1,
#         "addressdetails":1,
#         "format":"json",
#     }
#     url = "https://nominatim.openstreetmap.org/search?" + urlencode(params)
#     return url

# # lat, long, postal = getCoordinatesWithPostal("San Nicolas 2", "Bacoor", "Cavite","")
# location_unique = population_data.drop_duplicates(["Province","Municipality"])

# location_urls = {}
# for _, row in location_unique.sample(frac = 1).iterrows():
#     province = row["Province"]
#     municipality = row["Municipality"]
#     location_urls[f"{municipality},{province}"] = getCoordinatesWithPostal(province, municipality)

# print(location_urls)
    
# display(location_unique)
# test = location_unique.swifter.apply(
#     getCoordinatesWithPostal,
#     axis=1,
#     result_type="expand",
# )

# result = await asyncio.run(getHttpResults(location_urls))



 -->

In [5]:
if not os.path.exists(f"{raw_data_directory}/phl_admgz_adm01234.xlsx"):
    url = "https://data.humdata.org/dataset/caf116df-f984-4deb-85ca-41b349d3f313/resource/e74fd350-3728-427f-8b4c-0589dc563c87/download/phl_admgz_adm01234.xlsx"
    print(f"Downloading url {url}")
    wget.download(url, out=f"{os.getcwd()}/raw_data")

location_data = pd.read_excel(f"{raw_data_directory}/phl_admgz_adm01234.xlsx")
location_data = location_data[["ADM4_PCODE","ADM3_PCODE","ADM2_PCODE","ADM1_PCODE","ADM4_EN","ADM3_EN","ADM2_EN","ADM1_EN",]]
location_data.to_csv('partial_data/location_data.csv', index=False)

display(location_data.head(5))

c:\Python310\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,ADM4_PCODE,ADM3_PCODE,ADM2_PCODE,ADM1_PCODE,ADM4_EN,ADM3_EN,ADM2_EN,ADM1_EN
0,PH104210051,PH104210000,PH104200000,PH100000000,50th District (Pob.),Ozamis City,Misamis Occidental,Region X
1,PH082622001,PH082622000,PH082600000,PH080000000,A-et,Sulat,Eastern Samar,Region VIII
2,PH160205003,PH160205000,PH160200000,PH160000000,A. Beltran (Camalig),Jabonga,Agusan del Norte,Region XIII
3,PH034915001,PH034915000,PH034900000,PH030000000,A. Bonifacio,Llanera,Nueva Ecija,Region III
4,PH056203001,PH056203000,PH056200000,PH050000000,A. Bonifacio,Bulan,Sorsogon,Region V


### Obtaning Population Data
*String matching will be used for some attributes as the values are not exactly equal*
<!-- 
OLD CODE HERE:

location__muni_city = location_data.drop_duplicates(["ADM3_EN","ADM2_EN"])[["ADM2_EN","ADM3_EN",]]
population__muni_city = population_data.drop_duplicates(["Municipality","Province"])[["Municipality","Province",]]


location__muni_city["combined__cleaned"] = (location__muni_city["ADM3_EN"] + ", " + location__muni_city["ADM2_EN"])\
                        .str.replace("City of ","")\
                        .str.replace("City","")\
                        .str.replace("NCR", "National Capital Region")\
                        .str.replace("First District","")\
                        .str.replace("Second District","")\
                        .str.replace("Third District","")\
                        .str.replace("Fourth District","")

population__muni_city["combined__cleaned"]  = (population__muni_city["Municipality"] + ", " + population__muni_city["Province"])\
                            .str.replace("City of ","")\
                            .str.replace("City","")\
                            .str.replace("Of ","")\
                            .tolist()

population_combined = population__muni_city["combined__cleaned"].tolist()
for loc_add in location__muni_city["combined__cleaned"]:
    pop_add, score = process.extractOne(loc_add, population_combined)
    print("SCORE:",loc_add, " < --- Location| Population --- >", pop_add, score)
    if score >= 90:

        loc_muni = location__muni_city[ location__muni_city["combined__cleaned"] == loc_add].ADM2_EN.values[0]
        loc_prov = location__muni_city[ location__muni_city["combined__cleaned"] == loc_add].ADM3_EN.values[0]
        loc_brgys = location_data[ 
                    (location_data["ADM3_EN"] == loc_prov) &
                    (location_data["ADM2_EN"] == loc_muni)] \
                    .ADM4_EN\
                    .tolist()

        pop_muni = population__muni_city[ population__muni_city["combined__cleaned"] == pop_add].Municipality.values[0]
        pop_prov = population__muni_city[ population__muni_city["combined__cleaned"] == pop_add].Province.values[0]
        pop_brgys = population_data[
                    (population_data["Province"] == pop_prov) &
                    (population_data["Municipality"] == pop_muni)].Barangay \
                    .tolist()
        
        for loc_brgy in loc_brgys:
            pop_brgy, score = process.extractOne(loc_brgy, pop_brgys)

            population =  population_data[
                        (population_data["Barangay"] == pop_brgy) &
                        (population_data["Municipality"] == pop_muni) &
                        (population_data["Province"] == pop_prov)
                        ]["2020 Population"].values[0]
            location_data.loc[
                (location_data["ADM4_EN"] == loc_brgy) &
                (location_data["ADM3_EN"] == loc_prov) &
                (location_data["ADM2_EN"] == loc_muni),
                "population"
            ] = population
        population_combined.remove(pop_add)
        
    else:
        print("LOW SCORE:",loc_add, " < --- Location| Population --- >", pop_add, score)
        


# 20 MINUTES AND 38 SECS
 -->

In [6]:
location_data["combined__cleaned"] = (location_data["ADM3_EN"] + ", " + location_data["ADM2_EN"])\
                        .str.replace("City of ","")\
                        .str.replace("City","")\
                        .str.replace("NCR", "National Capital Region")\
                        .str.replace("First District","")\
                        .str.replace("Second District","")\
                        .str.replace("Third District","")\
                        .str.replace("Fourth District","")

population_data["combined__cleaned"]  = (population_data["Municipality"] + ", " + population_data["Province"])\
                            .str.replace("City of ","")\
                            .str.replace("City","")\
                            .str.replace("Of ","")\
                            .tolist()

prov_muni_keys = {} # key = location, value = popu
def find_match(row):
    filter = ''
    if row["combined__cleaned"] not in prov_muni_keys.keys():
        key, score = process.extractOne(row["combined__cleaned"], location_data["combined__cleaned"].unique())
        prov_muni_keys[row["combined__cleaned"]] = key
        filter = key
    else:
        filter = prov_muni_keys[row["combined__cleaned"]]

    location_data_filtered = location_data[location_data["combined__cleaned"] == filter]
    key1, score1, index1  = process.extractOne(row["Barangay"], location_data_filtered["ADM4_EN"])
    # print(row)
    # print(location_data.iloc[index1]["ADM4_PCODE"])
    return location_data.iloc[index1]["ADM4_PCODE"]

    #need i return ung brangay code ng LOCATION

population_data["brgy_code"] = population_data.apply(find_match, axis =1)

# Post processing
population_data["2020 Population"] = population_data["2020 Population"]/1000
population_data.drop(columns=["combined__cleaned"], axis =1, inplace=True)
location_data.drop(columns=["combined__cleaned"], axis =1, inplace=True)

population_data.to_csv('partial_data/population_data.csv', index=False)
display(population_data.head(5))

,Province,Municipality,Barangay,2020 Population,brgy_code
0,National Capital Region,"TONDO I/II, Manila",Barangay 1,2.995,PH133901001
1,National Capital Region,"TONDO I/II, Manila",Barangay 2,1.887,PH133901002
2,National Capital Region,"TONDO I/II, Manila",Barangay 3,1.341,PH133901003
3,National Capital Region,"TONDO I/II, Manila",Barangay 4,1.691,PH133901004
4,National Capital Region,"TONDO I/II, Manila",Barangay 5,1.288,PH133901005


### Downloading and Obtaining Language Data
Source: https://data.humdata.org/dataset/philippines-languages

In [7]:
def n_largest_prct(agg,n):
    # display(agg)
    return agg["percent_share"].nlargest(n).iloc[-1]

def n_largest_lang(agg,n):
    # print(agg["percent_share"].nlargest(n).iloc[-1])
    # print(agg[ agg["percent_share"] == agg["percent_share"].nlargest(n).iloc[-1] ] )
    return agg[ agg["percent_share"] == agg["percent_share"].nlargest(n).iloc[-1] ]["Language"].values[0]

In [8]:
languages_list = [
    "Adasen",
    "Agta",
    "Agta-Dumagat",
    "Agusan Manobo",
    "Agutaynen",
    "Akeanon",
    "Alangan",
    "Ata",
    "Ata-Manobo",
    "Ayangan Ifugao",
    "Ayta language group",
    "Bagobo/Tagabawa",
    "Balangao",
    "Baliwon/Ga'dang",
    "Banao",
    "Bantoanon",
    "Belwang (N.Bontok dialect)",
    "Bikol",
    "Binongan",
    "Binukid",
    "Bisaya/Binisaya",
    "B'laan/Blaan language group",
    "Bontok",
    "Bugkalot/Ilongot",
    "Buhid",
    "Cagayanen",
    "Capizeño",
    "Cebuano",
    "Chavacano",
    "Cuyonon/Cuyonen",
    "Davaweño",
    "Dibabawon",
    "Dumagat/Remontado",
    "English",
    "Gaddang",
    "Giangan",
    "Hanunuo",
    "Higaonon",
    "Hiligaynon Ilonggo",
    "Ibaloi/Ibaloy",
    "Ibanag",
    "Ibatan",
    "Ilianen Manobo",
    "Ilocano",
    "Iranun",
    "Iraya",
    "Isinai",
    "Isnag",
    "Itawis",
    "Ivatan",
    "Iwak/Iowak/Owak/I-wak",
    "Jama Mapun",
    "Kalagan",
    "Kalanguya",
    "Kalibugan/Kolibugan",
    "Kalinga language group",
    "Kamiguin",
    "Kankanaey",
    "Kapampangan",
    "Karao",
    "Karay-a",
    "Kirenteken",
    "Mabaka",
    "Maeng",
    "Maguindanao",
    "Majokayong",
    "Mamanwa",
    "Mandaya",
    "Manobo",
    "Manobo-Cotabato",
    "Mansaka",
    "Maranao",
    "Masadiit",
    "Masbateño/Masbatenon",
    "Matigsalog/Matigsalug",
    "Molbog",
    "Muyadan",
    "Obo Manobo",
    "Unspecified Sama language",
    "Palawani",
    "Palawano language group",
    "Pangasinan/Panggalato",
    "Paranan",
    "Romblomanon",
    "Sama Bangingi",
    "Sama Laut",
    "Sangil",
    "Subanen/Subanon/Subanun",
    "Surigaonon",
    "Tadyawan",
    "Tagakaulo",
    "Tagalog",
    "Tagbanua",
    "Tagbanua Calamian",
    "Tau-buid",
    "Tausug",
    "Tboli",
    "Teduray",
    "Tuwali",
    "Waray",
    "Yakan",
    "Yogad",
    "Zambal",
    "Other languages"
]

In [9]:
if not os.path.exists(f"{raw_data_directory}/ph_lang_admin2_v01.csv"):
    url = "https://data.humdata.org/dataset/4383caa9-b4e0-4608-80f6-2b74482fe8bd/resource/fb35553c-e3bc-4044-9729-8351ad4650f5/download/ph_lang_admin2_v01.csv"
    print(f"Downloading url {url}")
    wget.download(url, out=f"{os.getcwd()}/raw_data")

languages = pd.read_csv(f"{raw_data_directory}/ph_lang_admin2_v01.csv",encoding='latin-1', skiprows=[1])


languages_unpivot = pd.melt(languages, id_vars = ["admin2_name","admin2_pcode"], value_vars = languages_list, var_name="Language", value_name="percent_share")

languages_top = languages_unpivot.groupby(["admin2_name","admin2_pcode"]).apply(lambda agg: pd.Series({
    ("primary_language"):n_largest_lang(agg,1),
    ("primary_language_share"):n_largest_prct(agg,1),
    ("secondary_language"):n_largest_lang(agg,2),
    ("secondary_language_share"):n_largest_prct(agg,2),
    ("tertiary_language"):n_largest_lang(agg,3),
    ("tertiary_language_share"):n_largest_prct(agg,3),
})).reset_index()


Also includes: literacy rate for male, female, and both

In [10]:
languages_combined = languages_top\
                        .merge(languages[["admin2_name","literacy_all","literacy_male","literacy_female"]], on="admin2_name", how="left")\
                        .drop("admin2_pcode", axis=1)

languages_combined.to_csv('partial_data/languages_combined.csv', index=False)
display(languages_combined.head(5))

,admin2_name,primary_language,primary_language_share,secondary_language,secondary_language_share,tertiary_language,tertiary_language_share,literacy_all,literacy_male,literacy_female
0,PH012800000,Ilocano,0.981,Tagalog,0.010,Isnag,0.005,0.985,0.986,0.984
1,PH012900000,Ilocano,0.930,Other languages,0.033,Kankanaey,0.022,0.988,0.990,0.987
2,PH013300000,Ilocano,0.910,Tagalog,0.049,Kankanaey,0.023,0.988,0.989,0.987
3,PH015500000,Pangasinan/Panggalato,0.447,Ilocano,0.442,Tagalog,0.088,0.992,0.992,0.992
4,PH020900000,Ivatan,0.945,Tagalog,0.048,Ilocano,0.004,0.989,0.992,0.986


### Obtaining Consumer Price Index

Soure File: https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2M__PI__CPI__2018/0012M4ACP09.px/?rxid=ecb807b6-cb54-4d55-ad38-5fa91ed3643b

In [11]:
commodities = ['01 - FOOD AND NON-ALCOHOLIC BEVERAGES',
       '02 - ALCOHOLIC BEVERAGES AND TOBACCO',
       '03 - CLOTHING AND FOOTWEAR',
       '04 - HOUSING, WATER, ELECTRICITY, GAS, AND OTHER FUELS',
       '05 - FURNISHINGS, HOUSEHOLD EQUIPMENT AND ROUTINE HOUSEHOLD MAINTENANCE',
       '06 - HEALTH', '07 - TRANSPORT',
       '08 - INFORMATION AND COMMUNICATION',
       '09 - RECREATION, SPORT AND CULTURE', '10 - EDUCATION SERVICES',
       '11 - RESTAURANTS AND ACCOMMODATION SERVICES',
       '12 - FINANCIAL SERVICES',
       '13 - PERSONAL CARE, AND MISCELLANEOUS GOODS AND SERVICES']

In [12]:
cpi = pd.read_csv("raw_data/2M4ACP09.csv", skiprows=[0,1])
cpi["adm_level"] = cpi["Geolocation"].apply(lambda x: x.count('.'))
cpi = cpi[(cpi["adm_level"] == 4) | (cpi["Geolocation"] == "..National Capital Region (NCR)")]

cpi["Geolocation"] = cpi["Geolocation"].str.replace(".","", regex=False)
cpi = cpi[cpi["Geolocation"].str.startswith("City of") == False] # Removed component cities

cpi["Commodity Description"] = cpi["Commodity Description"].str.replace(r'\d{2} - ', '',regex=True)
cpi["Commodity Description"] = "CPI " + cpi["Commodity Description"]
cpi["Commodity Description"] = cpi["Commodity Description"].str.replace("AND ","").str.replace(" ","_").str.replace(",","").str.lower()
cpi_unpivot = cpi.pivot(index = ["Geolocation"], columns = "Commodity Description", values="2022 Ave").reset_index()

def getProvinceId(row):
       # special cases:
       if row == "Davao de Oro":
              row = "Compostela Valley"
       elif row == "National Capital Region (NCR)":
              return "PH133900000,PH137400000,PH137500000,PH137600000"
       key, _ = process.extractOne(row, location_data["ADM2_EN"].unique())
       return location_data.loc[location_data["ADM2_EN"] == key]["ADM2_PCODE"].values[0]

cpi_unpivot["province_id"] = cpi_unpivot["Geolocation"].apply(getProvinceId)

cpi_unpivot.to_csv('partial_data/cpi_unpivot.csv', index=False)
display(cpi_unpivot.head(5))

Commodity Description,Geolocation,cpi_alcoholic_beverages_tobacco,cpi_clothing_footwear,cpi_education_services,cpi_financial_services,cpi_food_non-alcoholic_beverages,cpi_furnishings_household_equipment_routine_household_maintenance,cpi_health,cpi_housing_water_electricity_gas_other_fuels,cpi_information_communication,cpi_personal_care_miscellaneous_goods_services,cpi_recreation_sport_culture,cpi_restaurants_accommodation_services,cpi_transport,province_id
0,Abra,155.858333,108.366667,105.400000,160.4,108.983333,110.750000,115.258333,119.825000,100.300000,109.691667,113.866667,134.891667,123.933333,PH140100000
1,Agusan del Norte,167.291667,104.133333,111.800000,146.0,115.141667,106.858333,121.450000,111.033333,100.650000,111.325000,111.825000,117.533333,118.633333,PH160200000
2,Agusan del Sur,152.516667,106.791667,120.350000,138.1,112.891667,112.533333,109.300000,116.191667,106.975000,109.316667,114.733333,124.733333,127.183333,PH160300000
3,Aklan,151.783333,105.100000,114.966667,145.5,108.491667,116.400000,113.266667,122.258333,102.825000,105.150000,106.408333,109.841667,120.725000,PH060400000
4,Albay,189.091667,123.775000,123.766667,146.0,114.341667,121.275000,122.716667,122.850000,103.508333,115.083333,113.925000,117.425000,151.541667,PH050500000


### Obtaining Gross Regional Domestic Product
https://openstat.psa.gov.ph/PXWeb/pxweb/en/DB/DB__2B__GP__RG__GRD/0012B5CPGD1.px/table/tableViewLayout1/?rxid=d8206526-ce02-4a4e-a0e8-11a051c34ea1

In [13]:
def getRegionalId(row):
       # inconsistent data cases:
       if "MIMAROPA" in row:
              row = "Region IV-B"
       elif "National Capital Region" in row:
              return "PH133900000,PH137400000,PH137500000,PH137600000"
  
       key, _ = process.extractOne(row, location_data["ADM1_EN"].unique())
       return location_data.loc[location_data["ADM1_EN"] == key]["ADM1_PCODE"].values[0]

In [14]:
grdp = pd.read_csv("raw_data/2B5CPGD1.csv", skiprows=[0,1])
grdp["region_id"] = grdp["Region"].apply(getRegionalId)

grdp["region_id"] = grdp["region_id"].str.split(",")
grdp = grdp.explode("region_id").drop("Region", axis=1)

grdp.to_csv('partial_data/grdp.csv', index=False)
display(grdp.head(5))

,At Current Prices 2022,region_id
0,6888962938,PH133900000
0,6888962938,PH137400000
0,6888962938,PH137500000
0,6888962938,PH137600000
1,365945436,PH140000000


### Obtaining Transportation Data

In [15]:
transpo_region_mappings = {
    "Autonomous Region in Muslim Mindanao":"BARMM",
    "Eastern Visayas":"Region VIII",
    "Ilocos Region":"Region I",
    "Cagayan Valley":"Region II",
    "Central Luzon":"Region III",
    "National Capital Region":"National Capital Region",
    "CALABARZON":"Region IV-A",
    "MIMAROPA Region":"Region IV-B",
    "Bicol Region":"Region V",
    "Western Visayas":"Region VI",
    "Central Visayas":"Region VII",
    "Zamboanga Peninsula":"Region IX",
    "Northern Mindanao":"Region X",
    "Davao Region":"Region XI",
    "SOCCSKSARGEN":"Region XII",
    "Caraga":"Region XIII",
    "Cordillera Administrative Region":"Cordillera Administrative Region (CAR) ",
}

In [16]:
transpo = pd.read_csv("raw_data/2D4BAH00.csv", skiprows=[0,1])

transpo["adm_level"] = transpo["Geolocation"].apply(lambda x: x.count('.'))
transpo["indu_hierarchy"] = transpo["Industry Description"].apply(lambda x: x.count('.'))
transpo = transpo[((transpo["adm_level"] == 2) & (transpo["indu_hierarchy"] == 2)) | (transpo["Geolocation"] == "..National Capital Region (NCR)")]
transpo = transpo[["Geolocation","Industry Description","2019 Number of Establishments"]]

transpo["Geolocation"] = transpo["Geolocation"].str.replace(".","", regex=False)
transpo["Industry Description"] = transpo["Industry Description"].str.replace(".","", regex=False)
transpo["2019 Number of Establishments"] = transpo["2019 Number of Establishments"].replace(r"s|c", np.nan, regex=True).astype(float)

transpo_unpivot = transpo.pivot(index = ["Geolocation"], columns = "Industry Description", values="2019 Number of Establishments").reset_index()

transpo_unpivot["region_id"] = transpo_unpivot["Geolocation"].map(transpo_region_mappings).apply(getRegionalId)
transpo_unpivot = transpo_unpivot[[
                            "region_id",
                            "Inland water transport",
                            "Passenger air transport",
                            "Sea and coastal water transport",
                            "Transport via buses",
                            "Transport via railways"]]

transpo_unpivot.rename(
    {"Inland water transport":"regional_inland_water_transport_count",
    "Passenger air transport":"regional_air_transport_count",
    "Sea and coastal water transport":"regional_coastal_water_transport_count",
    "Transport via buses":"regional_bus_transport_count",
    "Transport via railways":"regional_railway_transport_count"},
    axis=1, inplace=True
)

transpo_unpivot["region_id"] = transpo_unpivot["region_id"].str.split(",")
transpo_unpivot = transpo_unpivot.explode("region_id")

transpo_unpivot.to_csv('partial_data/transpo_unpivot.csv', index=False)
display(transpo_unpivot)

Industry Description,region_id,regional_inland_water_transport_count,regional_air_transport_count,regional_coastal_water_transport_count,regional_bus_transport_count,regional_railway_transport_count
0,PH150000000,NaN,NaN,NaN,NaN,NaN
1,PH050000000,NaN,NaN,11.0,14.0,NaN
2,PH040000000,NaN,NaN,8.0,64.0,NaN
3,PH020000000,NaN,NaN,4.0,6.0,NaN
4,PH160000000,NaN,NaN,NaN,NaN,NaN
5,PH030000000,NaN,NaN,NaN,18.0,NaN
6,PH070000000,NaN,NaN,38.0,17.0,NaN
7,PH140000000,NaN,NaN,NaN,5.0,NaN
8,PH110000000,NaN,NaN,4.0,5.0,NaN
9,PH080000000,23.0,NaN,NaN,7.0,NaN


### Merging all data

In [17]:
def create_description(row):
    description = (
        f"**Barangay Overview**\n"
        f"Barangay: {row['Barangay']}\n"
        f"City/Municipality: {row['City/Municipality']}\n"
        f"Province: {row['Province']}\n"
        f"Region: {row['Region']}\n\n"
        
        f"**Population and Languages**\n"
        f"In the year 2020, {row['Barangay']} was home to a population of {row['2020 Population']}. "
        f"The primary language spoken by the residents was {row['primary_language']}, constituting approximately {row['primary_language_share']}% of the population. "
        f"In addition, {row['secondary_language']} was another prevalent language, spoken by about {row['secondary_language_share']}% of the residents.\n\n"
        
        f"**Economic Landscape**\n"
        f"The Gross Regional Domestic Product (GDP) of {row['Region']} experienced notable growth, reaching {row['grdp_2022']} in the year 2022. "
        f"This growth underscores the economic vitality of the barangay and its contributions to the larger region's economy.\n\n"
    )
    return description


In [18]:
final_df = location_data.merge(
            population_data, right_on="brgy_code", left_on="ADM4_PCODE", how="left"
            )[["ADM4_PCODE", "ADM4_EN", "ADM3_PCODE","ADM3_EN", "ADM2_PCODE","ADM2_EN", "ADM1_PCODE", "ADM1_EN", "2020 Population"]]

final_df.rename({
    "ADM4_PCODE": "barangay_id",
    "ADM4_EN":"Barangay",
    "ADM3_PCODE":"municipality_id",
    "ADM3_EN":"City/Municipality",
    "ADM2_PCODE":"province_id",
    "ADM2_EN":"Province",
    "ADM1_PCODE":"region_id",
    "ADM1_EN":"Region",
}, axis=1, inplace=True)

final_df = final_df.merge(languages_combined,
    left_on="province_id", right_on="admin2_name", how="left").drop("admin2_name", axis=1)

final_df = final_df.merge(grdp,
    on="region_id", how="left").rename({
        "At Current Prices 2022":"grdp_2022",
    }, axis=1)

final_df = final_df.merge(transpo_unpivot,
    left_on="region_id", right_on="region_id", how="left")

final_df = final_df.merge(cpi_unpivot,
    left_on="province_id", right_on="province_id", how="left").drop("Geolocation", axis=1)

final_df['description'] = final_df.apply(create_description, axis=1)

# removed ids to make it human readable
final_df.drop(columns=["barangay_id","municipality_id","province_id","region_id"], axis=1, inplace=True)

final_df.to_csv('final_data/location_source.csv', index=False)

display(final_df.columns)

Index(['Barangay', 'City/Municipality', 'Province', 'Region',
       '2020 Population', 'primary_language', 'primary_language_share',
       'secondary_language', 'secondary_language_share', 'tertiary_language',
       'tertiary_language_share', 'literacy_all', 'literacy_male',
       'literacy_female', 'grdp_2022', 'regional_inland_water_transport_count',
       'regional_air_transport_count',
       'regional_coastal_water_transport_count',
       'regional_bus_transport_count', 'regional_railway_transport_count',
       'cpi_alcoholic_beverages_tobacco', 'cpi_clothing_footwear',
       'cpi_education_services', 'cpi_financial_services',
       'cpi_food_non-alcoholic_beverages',
       'cpi_furnishings_household_equipment_routine_household_maintenance',
       'cpi_health', 'cpi_housing_water_electricity_gas_other_fuels',
       'cpi_information_communication',
       'cpi_personal_care_miscellaneous_goods_services',
       'cpi_recreation_sport_culture',
       'cpi_restaurants_acc